# Experimenting With APIs

In this notebook, we will be experimenting with the APIs and create helper functions.

In [1]:
# general imports
import time
import re
from collections import namedtuple

# my imports
from helpers import kube

## Kubernetes API

We use the kubernetes API to fetch the current status of each deployment and see how many instances they have.

In [2]:
# start watch thread
kube.start_watch_thread()
# print details of a deployment
print(kube.live_deployments['helloworld-go-00001-deployment'])
# fetch a list of all knative deployments
kube.get_knative_deployments()

starting watch thread...
{'kind': 'Deployment', 'replicas': 0, 'ready_replicas': 0, 'last_update': 1611285138.1170037}


[KnativeDeployment(deployment='helloworld-go-00001-deployment', name='helloworld-go', revision='00001', revision_num=1)]

## Deploying On Knative

In this section, we will develop the functionality to deploy a given workload to **knative** using the `kn` CLI.
The other options were using [CRDs](https://stackoverflow.com/questions/61384188/how-to-deploy-a-knative-service-with-kubernetes-python-client-library),
or using [kubectl apply](https://developers.redhat.com/coderland/serverless/deploying-serverless-knative#invoking_your_service_from_the_command_line)
but `kn` seems to be more powerful.

In [3]:
# making sure kn is set up correctly
!kn service ls

NAME            URL                                            LATEST                AGE   CONDITIONS   READY   REASON
helloworld-go   http://helloworld-go.default.kn.nima-dev.com   helloworld-go-00001   10h   3 OK / 3     True    
